In [1]:
import numpy as np
import pandas as pd
from datetime import datetime

import networkx as nx
from networkx.algorithms import bipartite, matching
import multiprocessing

In [2]:
class Node():
    def __init__(self, inter,ID):
        '''
        para::
            inter: the intersection where the node is located
        '''
        self.inter = inter
        self.id = ID

class Path():
    def __init__(self, nodelist):
        self.nodeList = nodelist
        self.len = len(nodelist)
    
    def append(self, node):
        self.nodeList.append(node)
        self.len += 1
    
    def travelTimePath(self):
        t = 0
        for i in range(self.len-1):
            t += travelTime(self.nodeList[i], self.nodeList[i+1])
        return t
        
class Trip():
    def __init__(self, oneTripData):
        '''
        parameters::
            o_inter: the intersection of the origin of this trip
            d_inter: the intersection of the destination of this trip
            o_time: the expected departure time of the trip
            d_time: the expected arrival time of the trip
        '''
        self.node_o = Node(oneTripData['ointer'], oneTripData.name)
        self.node_d = Node(oneTripData['dinter'], oneTripData.name)
        self.ointer = self.node_o.inter
        self.dinter = self.node_d.inter
        self.oTime = oneTripData['otime']
        self.dTime = oneTripData['dtime']
        self.id = oneTripData.name

# This function calculate the distance between the origin and destination of two trips 
# def odDistance(trip_1, trip_2):
#     '''
#     Input:
#         trip_1, trip_2: two instances of class Trip
#     Output:
#         oDis: the distance of these two trips' origin
#         dDis: the distance of these two trips' destination
#     return:
#         a tuple (oDis, dDis)
#     '''
#     o_a = trip_1.oLong - trip_2.oLong
#     o_b = trip_1.oLati - trip_2.oLati
#     oDis = 2*np.arcsin(np.sin(o_b/2)**2 
#                        + np.cos(trip_1.oLati)*np.cos(trip_2.oLati)*np.sin(o_a/2)**2)*6378.137
#     d_a = trip_1.dLong - trip_2.dLong
#     d_b = trip_1.dLati - trip_2.dLati
#     dDis = 2*np.arcsin(np.sin(d_b/2)**2 
#                        + np.cos(trip_1.dLati)*np.cos(trip_2.dLati)*np.sin(d_a/2)**2)*6378.137
#     return (oDis, dDis)

    
def travelTime(node_1, node_2):
    '''
    parameters::
        distanceMatrixDict:
        node_1:
        node_2:
    return:
        
    '''
    return distanceMatrixDict[(node_1.inter, node_2.inter)]['time']

def isSharable(Trip_1, Trip_2, timePenalty = 600, distancePenalty = 1.2, coveragePenalty = 0.6):
    '''
    Input:
        trip_1, trip_2: two instances of class Trip
    Output:
        
    return:
        
    '''
    if (Trip_1.oTime > Trip_2.oTime):
        trip_1 = Trip_2
        trip_2 = Trip_1
    else:
        trip_1 = Trip_1
        trip_2 = Trip_2
    # case 1
    if (trip_1.dTime > trip_2.dTime):
        if(trip_1.oTime + travelTime(trip_1.node_o, trip_2.node_o) < trip_2.oTime):
            path1 = Path([trip_1.node_o, trip_2.node_o, trip_2.node_d, trip_1.node_d])
            if(trip_1.dTime-timePenalty < trip_1.oTime+path1.travelTimePath() < trip_1.dTime):
                path2 = Path([trip_1.node_o, trip_2.node_o, trip_2.node_d])
                if(trip_1.dTime-timePenalty < trip_1.oTime+path2.travelTimePath() < trip_1.dTime):
                    if(path1.travelTimePath() < distancePenalty*travelTime(trip_1.node_o, trip_1.node_d)):
                        return True
                    else:
                        return False
                else:
                    return False
            else:
                return False
        else:
            return False
    else:
        if (trip_2.oTime < trip_1.dTime < trip_2.dTime):
            if(trip_1.oTime + travelTime(trip_1.node_o, trip_2.node_o) < trip_2.oTime):
                path1 = Path([trip_1.node_o, trip_2.node_o, trip_1.node_d])
                if(trip_1.dTime-timePenalty < trip_1.oTime+path1.travelTimePath() < trip_1.dTime):
                    path2 = Path([trip_1.node_o, trip_2.node_o, trip_1.node_d, trip_2.node_d])
                    if(trip_2.dTime-timePenalty < trip_1.oTime+path2.travelTimePath() < trip_2.dTime):
                        if(path1.travelTimePath() < distancePenalty*travelTime(trip_1.node_o, trip_1.node_d)):
                            path3 = Path([trip_2.node_o, trip_1.node_d, trip_2.node_d])
                            if(path3.travelTimePath() < distancePenalty*travelTime(trip_2.node_o, trip_2.node_d)):
                                if(travelTime(trip_1.node_o, trip_2.node_o) < coveragePenalty*travelTime(trip_1.node_o, trip_1.node_d)):
                                    return True
                            else:
                                return False
                        else:
                            return False
                    else:
                        return False
                else:
                    return False
            else:
                return False
        else:
            return False
    return False

class TripEdge():
    def __init__(self, pre_node, suc_node, weight):
        self.node_1 = pre_node
        self.node_2 = suc_node
        self.w = weight
        self.edge = (self.node_1, self.node_2, self.w)
        
def addTrip(i, tripData):
    trip_list = [Trip(tripData.iloc[i])]
    return trip_list
    
def generateTripList(tripData):
    cores = multiprocessing.cpu_count()
    pool = multiprocessing.Pool(processes = cores-2)
    trip_list = []
    for trips in pool.starmap(addTrip, zip(range(len(tripData)), [tripData]*len(tripData))):
        trip_list += trips
    pool.close()
    return trip_list

def addEdge(i,trip_list):
    trip_edges = []
    for j in range(i+1, len(trip_list)):
        if isSharable(trip_list[i], trip_list[j]):
            weight = 1
            trip_edges.append((trip_list[i].id,trip_list[j].id))
    return trip_edges

def generateTripEdgeList(trip_list):
    cores = multiprocessing.cpu_count()
    pool = multiprocessing.Pool(processes = cores-2)
    trip_edge_list = []
    for trip_edge in pool.starmap(addEdge,zip(range(len(trip_list)-1),[trip_list]*(len(trip_list)-1))):
        trip_edge_list += trip_edge
    pool.close()
    return trip_edge_list
                
class TripGraph():
    def __init__(self, trip_edge_list):
        self.trip_edge_list = trip_edge_list
        self.num_trips = len(trip_edge_list)
        self.g = nx.Graph()
        self.g.add_edges_from(self.trip_edge_list,weight=1)
        
    def addTripEdge(self, trip_edge):
        self.trip_edge_list.append(trip_edge)
        self.num_trips += 1

    def maxMatching(self):
        graphs = list(nx.connected_component_subgraphs(self.g))
        self.matches = {}
        for graph in graphs:
            self.matches.update(matching.max_weight_matching(graph))
        self.number_match = len(self.matches)
        return (self.number_match, self.matches)
    
    def minFleeting(self):
        return self.number_match

In [1]:
start = datetime.now()

matrixPath = r'./od_time.xlsx'
tripPath = r'./trip_list.xlsx'
tripData = pd.read_excel(tripPath)
tripData = tripData.set_index('trip_rowid')
distanceMatrix = pd.read_excel(matrixPath)
distanceMatrix = distanceMatrix.sort_values(by = 'time')
distanceMatrix = distanceMatrix.sort_values(by = 'o')
distanceMatrix = distanceMatrix.set_index(list(distanceMatrix.columns[:2]))
distanceMatrixDict = distanceMatrix.to_dict(orient='index')

NameError: name 'datetime' is not defined

In [ ]:
interval = 86400
tripdata = tripData[tripData['otime'] < interval*(0+1)]
tripdata = tripdata[interval*0 < tripdata['otime']]
print('=='*20)
pre = datetime.now()
print('Data Preprocessing Finished in: %.2f s' % (pre-start).total_seconds())

Data Preprocessing Finished in: 12.27 s


In [ ]:
trip_list = generateTripList(tripData)
print('=='*20)
tripTime = datetime.now()
print('Trip List Generated in: %.2f s' % (tripTime-pre).total_seconds())

Trip List Generated in: 7.25 s


In [ ]:
trip_edge_list = generateTripEdgeList(trip_list)
print('=='*20)
edgeTime = datetime.now()
print('Trip Edge List Generated in: %.2f s' % (edgeTime-tripTime).total_seconds())

Trip Edge List Generated in: 3209.32 s


In [ ]:
trip_graph = TripGraph(trip_edge_list)
graphTime = datetime.now()
print('=='*20)
print('Trip Graph Initialized in: %.2f s' % (graphTime-edgeTime).total_seconds())
(num_matches, matching) = trip_graph.maxMatching()
minFleeting = trip_graph.minFleeting()
print('=='*20)
print('The Optimal Result Found is: %d' % minFleeting)
end = datetime.now()
print('=='*20)
print('The Time Spent was: %.2f s' % (end-start).total_seconds())

Trip Graph Initialized in: 4.21 s


In [ ]:
len(trip_graph.g.nodes),len(trip_graph.g.edges)

In [ ]:
len(tripData), len(trip_list)

In [ ]:
matches

In [ ]:
nx.write_graphml(trip_graph.g,'./trip_graph.graphml')

In [ ]:
distanceMatrix['time'].describe()

In [ ]:
len(trip_edge_list)